In [14]:
# Importing libraries
import mne
import numpy as np
from scipy.io import loadmat

# Un-fucking .mat format

In [4]:
bigData = loadmat('allParticipantsEEGData13.mat')

In [22]:
bigData.keys()
data = bigData['bigData']
print_formatted_struct(data)

P001:
-train:
--R001:
---X:
----[Array of shape (114176, 15)]
---trial:
----[Array of shape (1, 20)]
---y:
----[Array of shape (1, 20)]
---fs:
----[Array of shape (1, 1)]
---classes:
----[Array of shape (1, 2)]
--R002:
---X:
----[Array of shape (112128, 15)]
---trial:
----[Array of shape (1, 20)]
---y:
----[Array of shape (1, 20)]
---fs:
----[Array of shape (1, 1)]
---classes:
----[Array of shape (1, 2)]
--R003:
---X:
----[Array of shape (113664, 15)]
---trial:
----[Array of shape (1, 20)]
---y:
----[Array of shape (1, 20)]
---fs:
----[Array of shape (1, 1)]
---classes:
----[Array of shape (1, 2)]
--R004:
---X:
----[Array of shape (112128, 15)]
---trial:
----[Array of shape (1, 20)]
---y:
----[Array of shape (1, 20)]
---fs:
----[Array of shape (1, 1)]
---classes:
----[Array of shape (1, 2)]
--R005:
---X:
----[Array of shape (112128, 15)]
---trial:
----[Array of shape (1, 20)]
---y:
----[Array of shape (1, 20)]
---fs:
----[Array of shape (1, 1)]
---classes:
----[Array of shape (1, 2)]
-

In [36]:
def get_data(data, subject, run, electrode):
    # Get the data
    subject = 'P{:03d}'.format(subject + 1)
    run = 'R{:03d}'.format(run + 1)
    signal = data[subject][0,0]['train'][0,0][run][0,0]['X'][0,0][:,electrode]
    labels = data[subject][0,0]['train'][0,0][run][0,0]['y'][0,0][0,:]
    trial = data[subject][0,0]['train'][0,0][run][0,0]['trial'][0,0][0,:]
    fs = data[subject][0,0]['train'][0,0][run][0,0]['fs'][0,0]
    classes = data[subject][0,0]['train'][0,0][run][0,0]['classes'][0,0][0,:]
    return signal, labels, trial, fs, classes

def splitt_by_class(data, subject, run, electrode, max_len = -1, seconds_of_pre_trial_data=3):
    """
    Split the data in two classes: right hand and feet.
    y=1 -> right hand
    y=2 -> feet

    seconds_of_pre_trial_data: number of seconds of data to include before the trial starts, this is a sort of baseline.
    """
    signal, labels, trial, fs, _ = get_data(data, subject, run, electrode)

    if seconds_of_pre_trial_data*fs > trial[0]:
        print('seconds_of_pre_trial_data is too big, it must be smaller than the first trial.')
        return None

    # Split the data
    right_hand_seqences = []
    feet_seqences = []

    max_length = max([trial[j+1]-trial[j] for j in range(len(trial)-1)]) if max_len == -1 else max_len

    for i in range(len(trial)):
        current_trial = trial[i]
        next_trial = -1 if i == len(trial)-1 else trial[i+1]
        length = next_trial - current_trial if next_trial != -1 else max_length

        signal_trial_piece = signal[current_trial-fs*seconds_of_pre_trial_data:current_trial+length]
        if length < max_length: ## Zero pad the signal
            signal_trial_piece = np.append(signal_trial_piece, np.zeros(max_length - length))

        if labels[i] == 1:
            right_hand_seqences.append(signal_trial_piece)
        elif labels[i] == 2:
            feet_seqences.append(signal_trial_piece)
    return right_hand_seqences, feet_seqences
    
def avarage_run_in_frequency(data, subject):
    """
    Avarage the data in a given run. Data must be for one participant.
    """
    fs = data['train'][0,0]['R001'][0,0]['fs'][0,0]
    right_hand_seqences = []
    feet_seqences = []
    for electrode in range(data['train'][0,0]['R001'][0,0]['X'][0,0].shape[2]):  # For each electrode
        #pick the longest trial
        max_length = max([data['train'][0,0]['R{:03d}'.format(run + 1)][0,0]['trial'][0,0][:,electrode][j+1]-data['train'][0,0]['R{:03d}'.format(run + 1)][0,0]['trial'][0,0][:,electrode][j] for j in range(len(trial)-1) for run in range(len(data['train'][0,0].shape[1]))])
        #max_length = max([data['train'][0,0]['R{:03d}'.format(run + 1)][0,0]['X'][0,0][:,electrode] for run in range(data['train'][0,0].shape[1])])
        for run in range(data['train'][0,0].shape[1]): # For each run
            # split in epochs depending on hands or feet
            right_hand_seqences_new, feet_seqences_new = splitt_by_class(data, subject, run, electrode)
            if len(right_hand_seqences_new) < max_length:
                right_hand_seqences_new = np.append(right_hand_seqences_new, np.zeros(max_length - len(right_hand_seqences_new)))
            if len(feet_seqences_new) < max_length:
                feet_seqences_new = np.append(feet_seqences_new, np.zeros(max_length - len(feet_seqences_new)))

            right_hand_seqences.append(right_hand_seqences_new)
            feet_seqences.append(feet_seqences_new)

    # FFT of data
    runNum = 'R{:03d}'.format(run + 1)
    signal = data['train'][0,0][runNum][0,0]['X'][0,0][:,electrode]
    if signal.shape[0] < max_length: ## Zero pad the signal
        signal = np.append(signal, np.zeros(max_length - signal.shape[0]))
    fft_result = np.fft.fft(signal)
    # Create a frequency axis for the FFT)
    freq = np.fft.fftfreq(max_length, d=1/fs)


    return 0

R001:
-X:
--[Array of shape (114176, 15)]
-trial:
--[Array of shape (1, 20)]
-y:
--[Array of shape (1, 20)]
-fs:
--[Array of shape (1, 1)]
-classes:
--[Array of shape (1, 2)]
R002:
-X:
--[Array of shape (112128, 15)]
-trial:
--[Array of shape (1, 20)]
-y:
--[Array of shape (1, 20)]
-fs:
--[Array of shape (1, 1)]
-classes:
--[Array of shape (1, 2)]
R003:
-X:
--[Array of shape (113664, 15)]
-trial:
--[Array of shape (1, 20)]
-y:
--[Array of shape (1, 20)]
-fs:
--[Array of shape (1, 1)]
-classes:
--[Array of shape (1, 2)]
R004:
-X:
--[Array of shape (112128, 15)]
-trial:
--[Array of shape (1, 20)]
-y:
--[Array of shape (1, 20)]
-fs:
--[Array of shape (1, 1)]
-classes:
--[Array of shape (1, 2)]
R005:
-X:
--[Array of shape (112128, 15)]
-trial:
--[Array of shape (1, 20)]
-y:
--[Array of shape (1, 20)]
-fs:
--[Array of shape (1, 1)]
-classes:
--[Array of shape (1, 2)]
